In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
def extract_files(source_path, target_path):
    zip_ref = zipfile.ZipFile(source_path,'r')
    zip_ref.extractall(target_path)
    zip_ref.close()

In [ ]:
extract_files('/kaggle/input/dogs-vs-cats/test1.zip','/kaggle/working/')
extract_files('/kaggle/input/dogs-vs-cats/train.zip','/kaggle/working/')

In [ ]:
train_dir = "./train/"
test_dir  = "./test1/"

In [ ]:
filenames = os.listdir("/kaggle/working/train")
len(filenames)

In [ ]:
filenames = os.listdir("/kaggle/working/train")
base_path = '/kaggle/working/train/'
classes = []
images = []
for filename in tqdm(filenames):
    images.append(base_path + str(filename))
    classes.append(filename.split('.')[0])


df = pd.DataFrame({
    'filename': filenames,
    'class': classes
})

In [ ]:
df.head()

In [ ]:
df["class"] = df["class"].replace({0: 'cat', 1: 'dog'}) 
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
train_df, validation_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.head()

In [ ]:
train_df.reset_index(drop=True).head()

In [ ]:
validation_df.reset_index(drop=True).head()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
training_set = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col="filename",
    y_col="class",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
validation_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
validation_set = validation_datagen.flow_from_dataframe(
    validation_df,
    directory=train_dir,
    x_col="filename",
    y_col="class",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
def Model(input_shape=None, classes=None):

    img_input = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)
   
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(x)
    


    x = layers.Flatten(name='flatten')(x)
    x = layers.Dense(500, activation='relu', name='fc1')(x)
    x = layers.Dense(100, activation='relu', name='fc2')(x)
    x = layers.Dense(classes, activation='softmax', name='predictions')(x)

    model = tf.keras.Model(inputs=img_input, outputs=x, name="moldel")
    
    return model


In [ ]:
model = Model(input_shape=(224, 224,3), classes=2)
print(model.summary()) 

In [ ]:
model.compile(optimizer= tf.keras.optimizers.Adam(0.001), loss= 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
total_validate = len(validation_df)
total_train = len(train_df)
batch_size = 32
epochs=15
history = model.fit(
    training_set, 
    epochs=epochs,
    validation_data=validation_set,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,

)

# **Prediction part**

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_filenames = os.listdir("/kaggle/working/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
test_df.count()

In [ ]:
test_df


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_dataframe(
    test_df, 
    "/kaggle/working/test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(224, 224),
    batch_size=32,
    shuffle=False
)

In [ ]:
predict = model.predict(test_set, steps=np.ceil(nb_samples/32))

In [ ]:
predict[1]

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

In [ ]:
test_df['category'] = test_df['category'].apply(lambda x: 0 if x < 0.5 else 1)

In [ ]:
test_df

In [ ]:
test_df.to_csv("submission.csv")

In [ ]:
sample_test = test_df.head(15)
sample_test.head()


In [ ]:
from keras.preprocessing.image import  load_img

IMAGE_SIZE = 224
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = tf.keras.preprocessing.image.load_img("/kaggle/working/test1/"+str(filename))
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()

In [ ]:
filename = 'dog.6990.jpg'
category = 'dog'

image = tf.keras.preprocessing.image.load_img("/kaggle/working/train/"+str(filename))
plt.imshow(img)
plt.xlabel(filename + '(' + "{}".format(category) + ')' )
# plt.tight_layout()
plt.show()